In [1]:
from main import profile_run
from utillities import check_particle_collisions, execute_collision, run_loop, push_next_collision
%load_ext line_profiler

In [2]:
%lprun -f profile_run profile_run()

Placing particles
Finding inital collisions
Time elapsed: 24.103073120117188


Timer unit: 1e-06 s

Total time: 24.1964 s
File: /home/martin/repos/Exercises_CompPhys/ex1/main.py
Function: profile_run at line 77

Line #      Hits         Time  Per Hit   % Time  Line Contents
    77                                           def profile_run():
    78                                               # https://web.archive.org/web/20140513005858im_/http://www.appneta.com/blog/line-profiler-python/
    79         1          3.0      3.0      0.0      name = "profile_run"
    80         1       2996.0   2996.0      0.0      xi, N, T, R = read_params(para_dir + name)
    81         1         84.0     84.0      0.0      radii = np.ones(N) * R
    82         1         14.0     14.0      0.0      masses = np.ones(N)
    83         1          1.0      1.0      0.0      args = (N, T, radii, masses, xi)
    84         1   24193313.0 24193313.0    100.0      particles, t = run_loop(random_dist, args)

In [3]:
%lprun -f run_loop profile_run()

Placing particles
Finding inital collisions
Time elapsed: 20.89854335784912


Timer unit: 1e-06 s

Total time: 20.3847 s
File: /home/martin/repos/Exercises_CompPhys/ex1/utillities.py
Function: run_loop at line 270

Line #      Hits         Time  Per Hit   % Time  Line Contents
   270                                           def run_loop(init, args, condition=None, n_check=np.inf, TC=False):
   271         1         11.0     11.0      0.0      tic = time.time()
   272         1    3590863.0 3590863.0     17.6      t, particles, collisions, last_collided = setup_loop(init, args)
   273         1          2.0      2.0      0.0      N, T, radii, masses, xi = args
   274                                           
   275         1          1.0      1.0      0.0      n = 0
   276         1         76.0     76.0      0.0      bar = Bar("running simulation", max=T)
   277    118019      75576.0      0.6      0.4      while n < T:
   278    118018    1236453.0     10.5      6.1          col = get_next_col(collisions)
   279    118018     128279.0      1.1      0.6       

In [4]:
%lprun -f execute_collision profile_run()

Placing particles
Finding inital collisions
Time elapsed: 21.47944951057434


Timer unit: 1e-06 s

Total time: 15.6422 s
File: /home/martin/repos/Exercises_CompPhys/ex1/utillities.py
Function: execute_collision at line 247

Line #      Hits         Time  Per Hit   % Time  Line Contents
   247                                           def execute_collision(n, t, particles, collisions, last_collided, args, col, TC):
   248     10000       9473.0      0.9      0.1      N, T, radii, masses, xi = args
   249     10000       6851.0      0.7      0.0      t_next, i, j, t_added, col_type  = col
   250     10000     238131.0     23.8      1.5      particles[n+1] = particles[n]
   251     10000      16266.0      1.6      0.1      dt = col[0] - t[n]
   252     10000      10300.0      1.0      0.1      t[n+1] = t_next
   253                                           
   254     10000       6091.0      0.6      0.0      if TC:
   255                                                   xi = tc_check(i, n, t, last_collided, xi)
   256                                             

In [5]:
%lprun -f push_next_collision profile_run()

Placing particles
Finding inital collisions
Time elapsed: 55.10578274726868


Timer unit: 1e-06 s

Total time: 27.3163 s
File: /home/martin/repos/Exercises_CompPhys/ex1/utillities.py
Function: push_next_collision at line 31

Line #      Hits         Time  Per Hit   % Time  Line Contents
    31                                           def push_next_collision(particles, n, i, t, collisions, radii):
    32     19409     127953.0      6.6      0.5      wall0 = check_wall_collison(particles[n, i, 0], particles[n, i, 2], radii[i])
    33     19409      65562.0      3.4      0.2      wall1 = check_wall_collison(particles[n, i, 1], particles[n, i, 3], radii[i])
    34     19409      39978.0      2.1      0.1      if wall0 != np.inf: heapq.heappush(collisions, (t+wall0, i, -1, t, "wall0"))
    35     19409      31211.0      1.6      0.1      if wall1 != np.inf: heapq.heappush(collisions, (t+wall1, i, -1, t, "wall1"))
    36     19409    4903724.0    252.7     18.0      dts, cond = check_particle_collisions(particles, n, i, radii)
    37  19409000   11030395.0      0.6  

In [6]:
%lprun -f check_particle_collisions profile_run()

Placing particles
Finding inital collisions
Time elapsed: 20.99497961997986


Timer unit: 1e-06 s

Total time: 4.59823 s
File: /home/martin/repos/Exercises_CompPhys/ex1/utillities.py
Function: check_particle_collisions at line 134

Line #      Hits         Time  Per Hit   % Time  Line Contents
   134                                           def check_particle_collisions(particles, n, i, radii):
   135     20379      26447.0      1.3      0.6      N = len(radii)
   136     20379     165690.0      8.1      3.6      mask = np.arange(N) != i # remove the particle we are checking against
   137     20379     574244.0     28.2     12.5      one = np.ones_like(particles[n, mask, 0])
   138     20379     124540.0      6.1      2.7      one_v = np.array([one, one]).T
   139     20379     214158.0     10.5      4.7      R = radii[i] * one + radii[mask]
   140     20379     784156.0     38.5     17.1      dx = particles[n, mask, :2] - particles[n, i, :2] * one_v
   141     20379     721300.0     35.4     15.7      dv = particles[n, mask, 2:] - particles[n, i, 2:] * one_v
